In [1]:
import sys
sys.path.insert(0, "../../")


In [2]:
import os 
import frontmatter
from dataclasses import dataclass, field

def read_frontmater(file_path):
    # Create temp file

    with open(test_file_path) as f:
        post = frontmatter.load(f)

    fm = {key: post.get(key) for key in post.keys()}
    fm.update({'urls': []})

    for kb in fm.get('linked_kbs'):
        # url = re.search("(?P<url>https?://[^\s]+)", kb).group("url")
        t = kb.split('](')
        t = t[0][1:]
        fm['urls'].append(t)

    del fm['linked_kbs']

    return fm


test_file_path = "kb_md/posts/adding_a_beast_mode_calculation_to_your_chart/index.qmd"

class PathExists(Exception):
    def __init__(self, file_path):
        super().__init__(f"path {file_path} does not exist")


if not os.path.exists(test_file_path):
    raise  PathExists(test_file_path)
    
test_fm = read_frontmater(test_file_path)

print(test_fm.keys())

test_fm['urls']

dict_keys(['title', 'categories', 'date', 'url', 'article_id', 'views', 'created_date', 'last updated', 'urls'])


['https://domo-support.domo.com/s/article/360042924094',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamoGAC',
 'https://domo-support.domo.com/s/article/360043429993',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZanAGAS/beast-mode',
 'https://domo-support.domo.com/s/article/360043428753',
 'https://domo-support.domo.com/s/article/360043429693',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC/release-notes',
 'https://domo-support.domo.com/s/knowledge-base/',
 'https://domo-support.domo.com/s/article/360042924034',
 'https://domo-support.domo.com/s/article/360042925494',
 'https://domo-support.domo.com/s/knowledge-base',
 'https://domo-support.domo.com/s/article/360043429913',
 'https://domo-support.domo.com/s/article/360043429933',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZanAGAS',
 'https://domo-support.domo.com/s/article/360043429953']

In [3]:
import datacrew.crawler.crawler as dcc
driver = dcc.driversetup(is_headless=False)


In [4]:
from selenium.webdriver.common.by import By

TOPIC_URL = "https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC/connecting-data-to-domo?language=en_US"
TOPIC_URL = 'https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US'

BASE_URL = "https://domo-support.domo.com/s/topic/"

test_page_source = dcc.pagesource(url=TOPIC_URL, driver=driver,
                                  element_type=By.CLASS_NAME, element_id="section-list-item"
                                  )

# test_page_source


💤 loading https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US 💤
Page https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US is loaded within 10 seconds.


In [5]:
import datacrew.crawler.article as dca

from pprint import pprint

BASE_TOPIC_URL = "https://domo-support.domo.com/s/topic/"
TOPIC_URL = 'https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US'

test_article = dca.Article_Category(url=TOPIC_URL,
                 driver=driver,
                 base_url=BASE_TOPIC_URL)

💤 loading https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US 💤
Page https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US is loaded within 10 seconds.
💤 loading https://domo-support.domo.com/s/topic/0TO5w000000ZaoQGAS 💤
Message: 
Stacktrace:
Backtrace:
	GetHandleVerifier [0x002FDCE3+50899]
	(No symbol) [0x0028E111]
	(No symbol) [0x00195588]
	(No symbol) [0x001C08F9]
	(No symbol) [0x001C0AFB]
	(No symbol) [0x001EF902]
	(No symbol) [0x001DB944]
	(No symbol) [0x001EE01C]
	(No symbol) [0x001DB6F6]
	(No symbol) [0x001B7708]
	(No symbol) [0x001B886D]
	GetHandleVerifier [0x00563EAE+2566302]
	GetHandleVerifier [0x005992B1+2784417]
	GetHandleVerifier [0x0059327C+2759788]
	GetHandleVerifier [0x00395740+672048]
	(No symbol) [0x00298872]
	(No symbol) [0x002941C8]
	(No symbol) [0x002942AB]
	(No symbol) [0x002871B7]
	BaseThreadInitThunk [0x767100F9+25]
	RtlGetAppContainerNamedObjectPath [0x77097BBE+286]
	RtlGetAppContainerNamedObjectPath [

In [16]:
# test_article.__dict__.keys()
import pandas as pd

def test_children(dca_category : dca.Article_Category, results = [] ):

    # print(dca_category.category, dca_category.child_category_ls)
    
    results.append({
        'category' : dca_category.category,
        'url': dca_category.url
    })

    if not dca_category.child_category_ls:
        return results
    
    [test_children(child.get('child_article'), results) for child in dca_category.child_category_ls]

    return results


pd.DataFrame(test_children(test_article))


,category,url
0,Connectors,https://domo-support.domo.com/s/topic/0TO5w000...
1,API Connectors,https://domo-support.domo.com/s/topic/0TO5w000...
2,Partition Connectors,https://domo-support.domo.com/s/topic/0TO5w000...
3,Available Custom Connectors,https://domo-support.domo.com/s/topic/0TO5w000...
4,Database Connectors,https://domo-support.domo.com/s/topic/0TO5w000...
5,General Connect Info,https://domo-support.domo.com/s/topic/0TO5w000...
6,Writeback Connectors,https://domo-support.domo.com/s/topic/0TO5w000...
7,File Connectors,https://domo-support.domo.com/s/topic/0TO5w000...


In [7]:
import os

kb_path = "nbs/projects/kb_md/posts"
kb_path = "kb_md/posts/"

if not os.path.exists(kb_path):
    raise Exception('kb_path does not exist')

for path, subdirs, files in os.walk(kb_path):
    for name in files:
        if name != 'index.qmd':
            continue
        # print(path, name)
        file_path = os.path.join(path, name)
